### importing all the neccesary liberaries 

In [ ]:
import requests 
from bs4 import BeautifulSoup as bs
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.support.select import Select
import time 
from selenium.webdriver.common.by import By

### connecting to the driver using selenium

In [ ]:
driver = webdriver.Chrome()

### getting the link to the data 

In [ ]:
url = 'https://www.ncsl.org/technology-and-communication/state-9-1-1-bill-tracking-database'
driver.get(url)

### databases name

In [ ]:
# select all topics
#topic  = driver.find_element(By.ID , 'dnn_ctr15582_StateNetDB_ckBxTopics_7')
#topic.click()
# select all states
#allstates  = driver.find_element(By.ID , 'dnn_ctr15582_StateNetDB_ckBxAllStates')
#allstates.click()
# select the year 
#choose_year = input('please enter the year')
#select  = Select(driver.find_element(By.ID,'dnn_ctr15582_StateNetDB_ddlYear'))
#select.select_by_value('{}'.format(choose_year))

# search 
#search  = driver.find_element(By.ID , 'dnn_ctr15582_StateNetDB_btnSearch')
#search.click()

database_name = '9-1-1'

### getting all the html 

In [ ]:
soup = bs(driver.page_source, 'html.parser')

### searching for a speccific id 

In [ ]:
results = soup.find('div',{'id':'dnn_ctr16184_StateNetDB_linkList'})

### getting the bill number and bill link and year from the html

In [ ]:
atag = results.find_all('a')
tags = []
links = []
years = []
states = []
for tag in atag :
    
    #find bill number 
    tags.append(tag.text.strip())
    
    #find bill link
    links.append(tag['href'])
    
    #find years
    years.append(tag.find_next().find_next(text = True).replace('\n','').strip())
    
    #find the states
    state = tag.text[:2]
    states.append(state)

### getting the title of the bill and the status of every bill and the author 

In [ ]:
titles = []
statuses = []
authers = []
for tag in atag:
    
    # find the title of the bill
    title = tag.find_next('div').get_text()
    title = title.replace("\n",'')
    titles.append(title.strip())
    
    # find the status of the bill
    status = tag.find_next().find_next().find_next().find_next().find_next().find_next_sibling(text = True).replace('\n','')
    statuses.append(status.strip())
    
    #find the author of the bill
    auther = tag.find_next().find_next().find_next().find_next().find_next()\
    .find_next().find_next().find_next().find_next()\
    .find_next_sibling(text = True).replace('\n','')
    authers.append(auther.strip())
    

### finding supporting authors and additional bills if existed 

In [ ]:
# finding addtitional authers if existed 
substring  = 'Additional Authors'
substring_assosiated_bills = 'Associated Bills'
aditional_authors = []

# finding additional bills 
assosiated_bills = []
assosiated_bills_num = []
for tag in atag :
    string_we_need = tag.find_next().find_next().find_next().find_next().find_next()\
    .find_next().find_next().find_next().find_next().find_next().get_text().strip()
    
    string_for_ass = tag.find_next().find_next().find_next().find_next().find_next()\
    .find_next().find_next().find_next().find_next().find_next().find_next().find_next()\
    .find_next().get_text().strip()
    
    aditional_author = 0
    assosiated  = ''
    assosiated_num = 0
    
    if substring in string_we_need :
        aditional_author =  tag.find_next().find_next().find_next().find_next().find_next()\
        .find_next().find_next().find_next().find_next().find_next()\
        .find_next_sibling(text = True).replace('\n','').strip()
    
    if substring_assosiated_bills in string_for_ass:
        assosiated = tag.find_next().find_next().find_next().find_next().find_next()\
        .find_next().find_next().find_next().find_next().find_next().find_next().find_next()\
        .find_next().find_next_sibling(text =True).strip()
        assosiated_num = 1
        
    aditional_authors.append(aditional_author)
    assosiated_bills.append(assosiated)
    assosiated_bills_num.append(assosiated_num)
len(assosiated_bills_num)

### transforming the author type and author party

In [ ]:
# finding auther type
auther_party = []
auther_type = []
database = []
for i in authers:
    if '(' in i and ')' in i:
        auther_type.append(0)
    elif 'Office' in i :
        auther_type.append(2)
    else :
        auther_type.append(1)
        
        
    if '(R)' in i :
        auther_party.append(0)
    elif '(D)' in i or '(DFL)' in i or '(P)' in i :
        auther_party.append(1)
    elif '(I)' in i or '(CSI)' in i :
        auther_party.append(5)
    elif '(NP)' in i :
        auther_party.append(7)
    elif 'Office' in i or 'County' in i or 'Ethics & Campaign Disclosure Board' in i:
        auther_party.append(6)
    else :
        auther_party.append(8)
        
    database.append(database_name)


### transforming the status of the bill

In [ ]:
status_num = []
for i in statuses:
    if 'Adopted' in i :
        status_num.append(1)
    elif 'Enacted' in i :
        status_num.append(2)
    elif 'Failed - Adjourned' in i :
        status_num.append(4)
    elif 'Failed' in i:
        status_num.append(3)
    elif 'Override Pending' in i:
        status_num.append(5)
    elif 'Pending - Carryover' in i :
        status_num.append(7)
    elif 'Pending' in i :
        status_num.append(6)
    elif 'To Governor' in i :
        status_num.append(8)
    else:
        status_num.append(9)


### counting all the authors and additional authors 


In [ ]:
cosponsers = []
opposite = []
for ele , i in enumerate(aditional_authors):
    dem = 0
    rep = 0
    indp = 0
    if i != 0 :
        cosponsers.append(len(aditional_authors[ele].split(';')))
        for m in aditional_authors[ele].split(';') :
            if '(D)' in m or '(DFL)' in m or '(P)' in m :
                dem +=1
            elif '(R)' in m :
                rep +=1
            else:
                indp +=1
        if '(D)' in authers[ele] or '(DFL)' in authers[ele] or '(P)' in authers[ele]:
            opposite.append(rep+indp)
        elif '(R)' in authers[ele]:
            opposite.append(dem+indp)
        else:
            opposite.append(dem+indp+rep)
    else:
        cosponsers.append(0)
        opposite.append(0)


### using pandas we transformed into a dataframe

In [ ]:
dic = {
    'Database':database,
    'Year' : years,
    'Bill No': tags,
    'Title': titles,
    'Status': status_num,
    'status_say':statuses,
    'Author type':auther_type,
    'Author':authers,
    'Author party':auther_party,
    'Total number of cosponsors':cosponsers,
    'Total number of cosponsors from opposite party':opposite,
    'Associated bills':assosiated_bills_num,
    'Associated bill numbers':assosiated_bills,
    'link':links
}

### droped the status table

In [ ]:
df = pd.DataFrame(dic)
df.drop(columns = 'status_say', axis = 1 , inplace =True)

### changed the cosponsors accordingly to author party

In [ ]:
df.loc[df['Author party'] == 6 , 'Total number of cosponsors'] = 996
df.loc[df['Author party'] == 7 , 'Total number of cosponsors'] = 997
df.loc[df['Author party'] == 8 , 'Total number of cosponsors'] = 998
df.loc[df['Author party'] == 6 , 'Total number of cosponsors from opposite party'] = 996
df.loc[df['Author party'] == 7 , 'Total number of cosponsors from opposite party'] = 997
df.loc[df['Author party'] == 8 , 'Total number of cosponsors from opposite party'] = 998

In [ ]:
df.loc[df['Author party'] == 6]

### droped the bills from ppd and pnp

In [ ]:
portrico = df.loc[df['Author'].str.contains("(PPD)") | df['Author'].str.contains("(PNP)")]
df.drop(portrico.index , inplace = True)

In [ ]:
pd.set_option('display.max_rows', None)
df[['Year','Author type','Author','Author party','Total number of cosponsors','Total number of cosponsors from opposite party']]

In [ ]:
aditional_authors[3]

### uploaded to csv

In [ ]:
df.to_csv('{} {}.csv'.format(database_name,years[0]) , index = False)